# SRCNN

La superresolución de imágenes (SR) es un desafío fundamental en la visión por computadora, cuyo objetivo es reconstruir imágenes de alta resolución a partir de versiones de baja resolución. Este problema tiene aplicaciones clave en áreas como la medicina, la vigilancia, la fotografía y la restauración de imágenes históricas. Con el auge del Deep Learning, se han logrado avances significativos, permitiendo obtener resultados cada vez más precisos y realistas.

En este notebook, exploraremos uno de los enfoques más influyentes basados en redes neuronales profundas: la Super-Resolution Convolutional Neural Network ([SRCNN](https://arxiv.org/abs/1501.00092)). Este modelo aprende de extremo a extremo a transformar imágenes de baja resolución en imágenes de alta calidad, utilizando capas convolucionales para extraer características y reconstruir detalles perdidos, como se ilustra en el siguiente diagrama.

![SRCNN Architecture](https://raw.githubusercontent.com/jorge-jrzz/UEA-ML_SRCNN/refs/heads/main/imgs/srcnn-architecture.png)

Para entrenar nuestro modelo, utilizaremos el conjunto de datos [Dog and Cat Detection](https://www.kaggle.com/andrewmvd/dog-and-cat-detection) disponible en Kaggle. Descargaremos las imágenes mediante el módulo kagglehub, aunque también puedes obtenerlas manualmente desde el sitio web.

## Dependencias

In [ ]:
!wget https://raw.githubusercontent.com/jorge-jrzz/UEA-ML_SRCNN/refs/heads/main/install_datasets.py
!wget https://raw.githubusercontent.com/jorge-jrzz/UEA-ML_SRCNN/refs/heads/main/requirements.txt
%pip install -r requirements.txt --quiet

In [ ]:
from install_datasets import download_dataset

download_dataset("andrewmvd/dog-and-cat-detection")

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Aseguramos que la ruta donde se va a guardar el modelo en Google Drive existe:

In [ ]:
%%bash

mkdir -p /content/drive/MyDrive/super_resolution
cp -r training/* /content/drive/MyDrive/super_resolution

In [ ]:
MODEL_PATH = '/content/drive/MyDrive/super_resolution/model.h5'

In [ ]:
from glob import glob
from tqdm import tqdm
from pathlib import Path

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras import Model
from keras.losses import mse
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.utils import Sequence, plot_model
from keras.layers import Input, Conv2D, ReLU
from keras.preprocessing.image import img_to_array, load_img
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

## Definición de Parámetros para la Extracción de Parches

- **SCALE (2.0)**: Define el factor de escala para la super-resolución. Un valor de 2.0 significa que intentamos mejorar la resolución por un factor de 2.
  
- **INPUT_DIM (33)**: Tamaño de los parches de entrada que alimentaremos a la red. Estos parches son extraídos de las imágenes de baja resolución.
  
- **LABEL_SIZE (21)**: Tamaño de los parches objetivo (ground truth). Son más pequeños que los de entrada debido a que la red reduce el tamaño durante el procesamiento.
  
- **PAD (6)**: Representa el padding necesario para alinear correctamente los parches de entrada y salida. Se calcula como (INPUT_DIM - LABEL_SIZE) / 2.
  
- **STRIDE (14)**: Determina cuántos píxeles nos desplazamos al extraer parches consecutivos. Un valor menor crea más parches con mayor solapamiento.

- **SUBSET_SIZE (500)**: Número de imagenes que utilizaremos para entrenar y validar la red.

- **BATCH_SIZE (1024)**: Número de parches que procesamos en cada iteración del entrenamiento, basandose en la memoria disponible.

- **EPOCHS (12)**: Número de veces que recorremos el conjunto de datos durante el entrenamiento.

- **VALIDATION_SPLIT (0.1)**: Proporción de los datos que utilizaremos para la validación durante el entrenamiento.

In [ ]:
SCALE = 2.0             # Factor de escala para la super-resolución
INPUT_DIM = 33          # Tamaño de los parches de entrada (33x33 píxeles)
LABEL_SIZE = 21         # Tamaño de los parches de salida (21x21 píxeles)
PAD = int((INPUT_DIM - LABEL_SIZE) / 2.0)  # Padding necesario (6 píxeles)
STRIDE = 14             # Paso para la ventana deslizante (14 píxeles)
SUBSET_SIZE = 500       # Tamaño del subconjunto de datos a generar
BATCH_SIZE = 1024       # Tamaño del lote para el entrenamiento
EPOCHS = 12             # Número de épocas de entrenamiento
VALIDATION_SPLIT = 0.1  # 10% de los datos para validación

## Preparación de Imágenes para Super-Resolución

En el proceso de super-resolución, necesitamos trabajar con pares de imágenes: una versión de baja resolución (entrada) y una versión de alta resolución (objetivo). A continuación, se implementan varias funciones para preparar los datos de entrenamiento.

### Función: `reduce_image()`

Redimensiona una imagen según un factor de escala dado.

**Args**:
- image_array: Array NumPy que representa la imagen
- factor: Factor de escala para redimensionar (ej. 0.5 para reducir a la mitad, 2.0 para duplicar)

**Returns**:

Array NumPy con la imagen redimensionada

In [ ]:
def resize_image(image_array, factor):
    # Convertimos el array de NumPy a una imagen PIL
    original_image = Image.fromarray(image_array)
    
    # Calculamos el nuevo tamaño multiplicando las dimensiones originales por el factor
    new_size = np.array(original_image.size) * factor
    new_size = new_size.astype(np.int32)  # Convertimos a enteros
    new_size = tuple(new_size)  # Convertimos a tupla para resize()
    
    # Redimensionamos la imagen
    resized = original_image.resize(new_size)
    
    # Convertimos de vuelta a array y aseguramos que sea uint8
    resized = img_to_array(resized)
    resized = resized.astype(np.uint8)
    
    return resized

### Función: `downsize_upsize_image()`

Simula el proceso de degradación de resolución reduciendo y luego aumentando el tamaño de una imagen. Este proceso es fundamental para generar pares de entrenamiento para super-resolución.

**Args**:
- image: Imagen original de alta resolución
- scale: Factor de escala para la reducción/ampliación

**Returns**:

Imagen de "baja resolución" del mismo tamaño que la original

In [ ]:
def downsize_upsize_image(image, scale):
    # Primero reducimos la imagen (downsampling)
    scaled = resize_image(image, 1.0 / scale)
    
    # Luego la ampliamos de nuevo al tamaño original (upsampling)
    # Esto crea una versión de menor calidad pero mismo tamaño
    scaled = resize_image(scaled, scale / 1.0)
    
    return scaled

### Función: `tight_crop_image()`

Recorta una imagen para asegurar que sus dimensiones sean divisibles por el factor de escala. Esto es necesario para que el proceso de extracción de parches funcione correctamente.

**Args**:
- image: Imagen a recortar
- scale: Factor de escala

**Returns**:

Imagen recortada con dimensiones divisibles por scale

In [ ]:
def tight_crop_image(image, scale):
    height, width = image.shape[:2]
    
    # Ajustamos el ancho y alto para que sean divisibles por el factor de escala
    width -= int(width % scale)
    height -= int(height % scale)
    
    # Devolvemos la imagen recortada
    return image[:height, :width]

### Funciones para Extraer Parches de Entrenamiento

#### Función: `crop_input()`

Extrae un parche de la imagen de entrada en la posición (x,y).
    
**Args**:
- image: Imagen de entrada (baja resolución)
- x, y: Coordenadas de la esquina superior izquierda del parche

**Returns**:

Parche de tamaño INPUT_DIM x INPUT_DIM

#### Función:`crop_output()`

Extrae un parche de la imagen objetivo (alta resolución) en la posición (x,y), considerando el padding necesario para alinear con la salida de la red.

**Args**:
- image: Imagen objetivo (alta resolución)
- x, y: Coordenadas de la esquina superior izquierda del parche

**Returns**:

Parche de tamaño LABEL_SIZE x LABEL_SIZE

In [ ]:
def crop_input(image, x, y):
    y_slice = slice(y, y + INPUT_DIM)
    x_slice = slice(x, x + INPUT_DIM)
    return image[y_slice, x_slice]

def crop_output(image, x, y):
    # Añadimos PAD para compensar la reducción de tamaño en la red
    y_slice = slice(y + PAD, y + PAD + LABEL_SIZE)
    x_slice = slice(x + PAD, x + PAD + LABEL_SIZE)
    return image[y_slice, x_slice]

### Carga y Preparación del Conjunto de Datos

In [ ]:
# Obtenemos las rutas de todas las imágenes en el directorio
file_patten = (Path('/content') / 'images' / '*.png')
file_pattern = str(file_patten)
dataset_paths = [*glob(file_pattern)]

# Para reducir el tiempo de entrenamiento, seleccionamos un subconjunto aleatorio
dataset_paths = np.random.choice(dataset_paths, SUBSET_SIZE)

#### Visualización de una Imagen de Ejemplo

In [ ]:
# Mostramos una imagen aleatoria del conjunto de datos para verificar
path = np.random.choice(dataset_paths)
img = plt.imread(path)
plt.imshow(img)

### Generación de Parches para Entrenamiento

In [ ]:
# Creamos directorios para almacenar los parches
%%bash

mkdir -p data
mkdir -p training

In [ ]:
# Procesamos cada imagen para generar parches de entrenamiento
for image_path in tqdm(dataset_paths):
    # Obtenemos el nombre base del archivo
    filename = Path(image_path).stem
    
    # Cargamos la imagen y la convertimos a un array NumPy
    image = load_img(image_path)
    image = img_to_array(image)
    image = image.astype(np.uint8)
    
    # Recortamos la imagen para que sus dimensiones sean divisibles por SCALE
    image = tight_crop_image(image, SCALE)
    
    # Generamos la versión de baja resolución
    scaled = downsize_upsize_image(image, SCALE)
    
    # Obtenemos las dimensiones de la imagen
    height, width = image.shape[:2]
    
    # Extraemos parches deslizando una ventana por la imagen
    for y in range(0, height - INPUT_DIM + 1, STRIDE):
        for x in range(0, width - INPUT_DIM + 1, STRIDE):
            # Extraemos el parche de entrada (baja resolución)
            crop = crop_input(scaled, x, y)
            
            # Extraemos el parche objetivo correspondiente (alta resolución)
            target = crop_output(image, x, y)
            
            # Guardamos los parches como archivos NumPy
            np.save(f'data/{filename}_{x}_{y}_input.np', crop)
            np.save(f'data/{filename}_{x}_{y}_output.np', target)

### Cargador de Datos para Entrenamiento

Como no podemos mantener todos los parches en memoria simultáneamente, los hemos guardado en disco. Ahora necesitamos un cargador de datos que lea estos parches en lotes durante el entrenamiento. Implementaremos esto mediante la clase `PatchesDataset`, que hereda de `Sequence` de Keras para permitir la carga eficiente de datos.

In [ ]:
class PatchesDataset(Sequence):
    def __init__(self, batch_size, *args, **kwargs):
        self.batch_size = batch_size
        
        # Obtenemos las rutas de todos los archivos de entrada y salida
        self.input = [*glob('data/*_input.np.npy')]
        self.output = [*glob('data/*_output.np.npy')]
        
        # Ordenamos las listas para asegurar la correspondencia entre entrada y salida
        self.input.sort()
        self.output.sort()
        
        # Guardamos el número total de muestras
        self.total_data = len(self.input)

    def __len__(self):
        return int(np.ceil(self.total_data / self.batch_size))

    def __getitem__(self, idx):
        """
        Obtiene un lote de datos.
        
        Args:
            idx: Índice del lote
        
        Returns:
            Tupla (entradas, salidas) con los datos del lote
        """
        # Calculamos los índices de inicio y fin para este lote
        batch_start = idx * self.batch_size
        batch_end = min((idx + 1) * self.batch_size, self.total_data)
        
        # Inicializamos arrays para almacenar los datos del lote
        batch_input = []
        batch_output = []
        
        # Cargamos cada muestra del lote
        for i in range(batch_start, batch_end):
            # Cargamos el parche de entrada
            input_data = np.load(self.input[i])
            input_data = input_data.astype(np.float32) / 255.0  # Normalizamos a [0,1]
            batch_input.append(input_data)
            
            # Cargamos el parche de salida correspondiente
            output_data = np.load(self.output[i])
            output_data = output_data.astype(np.float32) / 255.0  # Normalizamos a [0,1]
            batch_output.append(output_data)
        
        # Convertimos las listas a arrays NumPy
        return np.array(batch_input), np.array(batch_output)

Crea una instancia del generador de datos:

In [ ]:
train_ds = PatchesDataset(BATCH_SIZE)
len(train_ds)

Visualizar la forma (dimensiones) de los lotes de entrada y salida:

In [ ]:
input, output = train_ds[0]
input.shape, output.shape

## Modelo SRCNN

La arquitectura de SRCNN es elegante en su simplicidad pero poderosa en sus resultados. El modelo consta de tres capas convolucionales, cada una con un propósito específico en el proceso de super-resolución:

1. **Primera capa (Extracción de parches)**: Utiliza 64 filtros con un tamaño de kernel de 9×9 para extraer parches de baja resolución y representarlos como mapas de características de alta dimensión.

2. **Segunda capa (Mapeo no lineal)**: Emplea 32 filtros con un tamaño de kernel de 1×1 para mapear los mapas de características de alta dimensión a mapas de características que representan parches de alta resolución.

3. **Tercera capa (Reconstrucción)**: Utiliza filtros con un tamaño de kernel de 5×5 para combinar las predicciones de los parches superpuestos, reconstruyendo la imagen de alta resolución final.

Esta arquitectura implementa el enfoque de aprendizaje de extremo a extremo para la super-resolución, donde la red aprende directamente la transformación de imágenes de baja resolución a alta resolución.

### Parámetros del Modelo SRCNN

- **Inicialización de pesos**: Utilizamos la inicialización 'he_normal' (también conocida como inicialización de He), que es especialmente adecuada para capas con activación ReLU. Esta inicialización ayuda a evitar el problema de desvanecimiento del gradiente.

- **Función de activación ReLU**: Aplicamos la función de activación Rectified Linear Unit (ReLU) después de las dos primeras capas convolucionales. ReLU introduce no-linealidad en el modelo y ayuda a acelerar la convergencia durante el entrenamiento.

- **Tamaños de kernel**:
  - Kernel 9×9 en la primera capa: Permite capturar un contexto espacial más amplio para la extracción de características.
  - Kernel 1×1 en la segunda capa: Realiza una transformación no lineal punto a punto sin mezclar información espacial.
  - Kernel 5×5 en la tercera capa: Proporciona suficiente contexto para reconstruir detalles finos en la imagen final.

- **Número de filtros**:
  - 64 filtros en la primera capa: Permite extraer un conjunto rico de características de bajo nivel.
  - 32 filtros en la segunda capa: Reduce la dimensionalidad mientras mantiene la información relevante.
  - Mismo número de filtros que canales de entrada en la capa final: Garantiza que la salida tenga la misma profundidad que la imagen original.

#### Función:`crop_output()`

Crea un modelo SRCNN (Super-Resolution Convolutional Neural Network).
    
**Args**:

- height: Altura de la imagen de entrada
- width: Ancho de la imagen de entrada
- depth: Profundidad (canales) de la imagen de entrada
    
**Returns**:

Modelo SRCNN compilado

In [ ]:
def create_model(height, width, depth):
    # Capa de entrada
    input = Input(shape=(height, width, depth))
    
    # Primera capa convolucional - Extracción de parches
    # 64 filtros con kernel 9x9 para extraer características de bajo nivel
    x = Conv2D(filters=64, kernel_size=(9, 9), kernel_initializer='he_normal')(input)
    x = ReLU()(x)  # Activación ReLU para introducir no-linealidad
    
    # Segunda capa convolucional - Mapeo no lineal
    # 32 filtros con kernel 1x1 para mapear características a representaciones de alta resolución
    x = Conv2D(filters=32, kernel_size=(1, 1), kernel_initializer='he_normal')(x)
    x = ReLU()(x)  # Activación ReLU
    
    # Tercera capa convolucional - Reconstrucción
    # Filtros con kernel 5x5 para reconstruir la imagen final de alta resolución
    output = Conv2D(filters=depth, kernel_size=(5, 5), kernel_initializer='he_normal')(x)
    
    # Creamos y devolvemos el modelo
    return Model(input, output)

### Función de Pérdida y Optimizador

- **Función de pérdida - Error Cuadrático Medio (MSE)**: 
  El MSE es una elección común para problemas de regresión como la super-resolución. Calcula el promedio de los cuadrados de las diferencias entre los valores predichos y los valores reales. En el contexto de imágenes, mide la diferencia de intensidad de píxeles entre la imagen reconstruida y la imagen objetivo de alta resolución.

- **Optimizador - Adam**:
  Adam (Adaptive Moment Estimation) es un algoritmo de optimización que combina las ventajas de los algoritmos AdaGrad y RMSProp. Adapta las tasas de aprendizaje para cada parámetro, lo que lo hace eficiente para problemas con gradientes dispersos o ruidosos, como es común en el procesamiento de imágenes.

- **Tasa de aprendizaje**:
  Utilizamos una tasa de aprendizaje de 0.0003, que es un valor equilibrado para SRCNN. Una tasa demasiado alta podría causar que el entrenamiento sea inestable, mientras que una tasa demasiado baja podría hacer que el entrenamiento sea innecesariamente lento.

In [ ]:
# Creamos el modelo SRCNN
model = create_model(INPUT_DIM, INPUT_DIM, 3)  # 3 canales para imágenes RGB

# Compilamos el modelo con el optimizador Adam y la función de pérdida de error cuadrático medio
model.compile(
    optimizer = Adam(learning_rate=1e-3, decay=1e-3 / EPOCHS),  # Tasa de aprendizaje ajustada para SRCNN
    loss='mse'  # Error cuadrático medio (Mean Squared Error)
)

# Mostramos un resumen de la arquitectura del modelo
display(model.summary())

# Visualizamos la arquitectura del modelo
try:
    plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
    display(Image.open('model.png'))
except Exception as e:
    print(f"No se pudo generar la visualización del modelo: {e}")

### Entrenamiento del Modelo SRCNN

El entrenamiento de un modelo de super-resolución como SRCNN requiere una cantidad significativa de datos y tiempo de cómputo. En este notebook, entrenaremos el modelo durante `<EPOCHS>` épocas, lo que debería ser suficiente para observar mejoras significativas en la calidad de las imágenes.

Durante el entrenamiento, monitorizaremos la pérdida tanto en el conjunto de entrenamiento como en el de validación. La pérdida de validación nos ayudará a determinar si el modelo está generalizando bien o si está sobreajustando los datos de entrenamiento.

Utilizamos el callback `ModelCheckpoint` para guardar automáticamente la mejor versión del modelo según la pérdida de validación. Esto nos asegura que, incluso si el modelo comienza a sobreajustar en épocas posteriores, conservaremos la versión con mejor rendimiento.

In [ ]:
# Configuramos los callbacks para el entrenamiento
callbacks = [
    # Guardamos el mejor modelo basado en la pérdida de validación
    ModelCheckpoint(
        filepath='training/srcnn_model_best.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

# Calculamos el número de muestras para validación
total_samples = len(train_ds.input)
val_samples = int(total_samples * VALIDATION_SPLIT)
train_samples = total_samples - val_samples

print(f"Total de muestras: {total_samples}")
print(f"Muestras de entrenamiento: {train_samples}")
print(f"Muestras de validación: {val_samples}")

In [ ]:
# Entrenamos el modelo
history = model.fit(
    train_ds,  # Generador de datos de entrenamiento
    epochs=EPOCHS,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks,
    verbose=1
)

# Guardamos el modelo final
model.save(MODEL_PATH)
print("Modelo guardado correctamente.")

## Visualización de Resultados del Entrenamiento

Para evaluar el progreso del entrenamiento, visualizaremos las curvas de pérdida tanto para el conjunto de entrenamiento como para el de validación. Estas gráficas nos ayudarán a entender:

1. Si el modelo está aprendiendo efectivamente (la pérdida disminuye con el tiempo)
2. Si el modelo está sobreajustando (la pérdida de validación aumenta mientras la de entrenamiento sigue disminuyendo)
3. Si el modelo ha convergido (la pérdida se estabiliza después de cierto número de épocas)

Además, visualizaremos algunos ejemplos de super-resolución aplicando nuestro modelo a imágenes de prueba para evaluar cualitativamente su rendimiento.

In [ ]:
# Visualizamos las curvas de pérdida
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.title('Curvas de Pérdida del Modelo SRCNN')
plt.xlabel('Época')
plt.ylabel('Error Cuadrático Medio (MSE)')
plt.legend()
plt.grid(True)
plt.show()

## Cargar el modelo desde  Google Drive

In [ ]:
model = load_model(MODEL_PATH, custom_objects={'mse': mse})

## Evaluación del Modelo SRCNN

Una vez entrenado nuestro modelo SRCNN, es fundamental evaluar su rendimiento para comprender qué tan bien realiza la tarea de super-resolución. En esta sección, evaluaremos el modelo de dos maneras:

1. **Evaluación cuantitativa**: Utilizaremos métricas objetivas como PSNR (Peak Signal-to-Noise Ratio) y SSIM (Structural Similarity Index) para medir numéricamente la calidad de las imágenes reconstruidas.

2. **Evaluación cualitativa**: Visualizaremos ejemplos de imágenes procesadas por nuestro modelo para realizar una evaluación subjetiva de la calidad visual.

Estas evaluaciones nos permitirán entender las fortalezas y limitaciones de nuestro modelo SRCNN.

### Métricas de Evaluación

Para evaluar objetivamente la calidad de las imágenes generadas por nuestro modelo SRCNN, utilizaremos dos métricas principales:

- **PSNR (Peak Signal-to-Noise Ratio)**: Mide la relación entre la potencia máxima posible de una señal y la potencia del ruido que afecta a su representación. En el contexto de imágenes, un valor más alto de PSNR generalmente indica una mejor calidad de reconstrucción. Se mide en decibelios (dB).

- **SSIM (Structural Similarity Index)**: A diferencia del PSNR, que se basa en el error cuadrático medio, el SSIM considera cambios en la estructura, luminancia y contraste. Produce valores entre -1 y 1, donde 1 indica una similitud perfecta entre las imágenes comparadas.

Estas métricas nos ayudarán a cuantificar la mejora que nuestro modelo SRCNN proporciona en comparación con métodos más simples como la interpolación bicúbica.

#### Función: `calculate_psnr()`

Calcula el Peak Signal-to-Noise Ratio entre dos imágenes.
    
**Args**:

- img1: Primera imagen (referencia)
- img2: Segunda imagen (comparación)
    
**Returns**:

Valor PSNR en decibelios (dB)

#### Función: `calculate_ssim()`

Calcula el Structural Similarity Index entre dos imágenes.

Args:
- img1: Primera imagen (referencia)
- img2: Segunda imagen (comparación)
- multichannel: Si es True, considera múltiples canales (RGB)

Returns:

Valor SSIM entre -1 y 1

In [ ]:
def calculate_psnr(img1, img2):
    # Aseguramos que las imágenes estén en el rango [0, 1]
    if img1.max() > 1.0:
        img1 = img1.astype(np.float32) / 255.0
    if img2.max() > 1.0:
        img2 = img2.astype(np.float32) / 255.0
        
    return psnr(img1, img2)

def calculate_ssim(img1, img2, multichannel=True):
    # Aseguramos que las imágenes estén en el rango [0, 1]
    if img1.max() > 1.0:
        img1 = img1.astype(np.float32) / 255.0
    if img2.max() > 1.0:
        img2 = img2.astype(np.float32) / 255.0
        
    return ssim(img1, img2, multichannel=multichannel)

### Función para Aplicar Super-Resolución a una Imagen: `apply_srcnn()`

Aplica el modelo SRCNN a una imagen para realizar super-resolución.
    
**Args**:
- model: Modelo SRCNN entrenado
- img: Imagen de entrada (alta resolución original)
- scale: Factor de escala para la super-resolución
        
**Returns**:

Tupla con (imagen original, imagen de baja resolución, imagen reconstruida por SRCNN)

In [ ]:
def apply_srcnn(model, img, scale=2.0):
    # Aseguramos que la imagen esté en el formato correcto
    if isinstance(img, str):
        img = load_img(img)
        img = img_to_array(img)
    
    # Recortamos la imagen para que sus dimensiones sean divisibles por scale
    img = tight_crop_image(img, scale)
    
    # Creamos la versión de baja resolución
    lr_img = downsize_upsize_image(img, scale)
    
    # Normalizamos la imagen para la predicción
    lr_img_norm = lr_img.astype(np.float32) / 255.0
    
    # Aplicamos el modelo para obtener la imagen de super-resolución
    sr_img = model.predict(np.expand_dims(lr_img_norm, axis=0))[0]
    
    # Convertimos de vuelta al rango [0, 255]
    sr_img = (sr_img * 255.0).astype(np.uint8)
    
    return img, lr_img, sr_img

Función para Visualizar Resultados `visualize_results()`

Visualiza los resultados de super-resolución para comparación.
    
**Args**:
- original: Imagen original de alta resolución
- bicubic: Imagen de baja resolución (interpolación bicúbica)
- srcnn: Imagen reconstruida por SRCNN
- title: Título opcional para la figura

In [ ]:
def visualize_results(original, bicubic, srcnn, title=None):
    # Configuramos la figura
    fig, axes = plt.subplots(1, 3, figsize=(20, 10))
    
    # Mostramos las imágenes
    axes[0].imshow(original)
    axes[0].set_title('Original (Alta Resolución)')
    axes[0].axis('off')
    
    axes[1].imshow(bicubic)
    axes[1].set_title('Bicúbica (Baja Resolución)')
    axes[1].axis('off')
    
    axes[2].imshow(srcnn)
    axes[2].set_title('SRCNN (Reconstruida)')
    axes[2].axis('off')
    
    # Calculamos y mostramos las métricas
    psnr_bicubic = calculate_psnr(original, bicubic)
    ssim_bicubic = calculate_ssim(original, bicubic)
    
    psnr_srcnn = calculate_psnr(original, srcnn)
    ssim_srcnn = calculate_ssim(original, srcnn)
    
    if title:
        plt.suptitle(f"{title}\n"
                    f"PSNR: Bicúbica={psnr_bicubic:.2f}dB, SRCNN={psnr_srcnn:.2f}dB | "
                    f"SSIM: Bicúbica={ssim_bicubic:.4f}, SRCNN={ssim_srcnn:.4f}", 
                    fontsize=16)
    
    plt.tight_layout()
    plt.show()
    
    return psnr_bicubic, ssim_bicubic, psnr_srcnn, ssim_srcnn

### Evaluación en Imágenes de Prueba

Evaluaremos nuestro modelo SRCNN en algunas imágenes de prueba para ver cómo se desempeña en la tarea de super-resolución. Seleccionaremos algunas imágenes aleatorias de nuestro conjunto de datos y compararemos:

1. La imagen original de alta resolución
2. La versión de baja resolución obtenida mediante interpolación bicúbica
3. La imagen reconstruida por nuestro modelo SRCNN

Para cada comparación, calcularemos las métricas PSNR y SSIM para cuantificar la mejora.

In [ ]:
# Seleccionamos algunas imágenes aleatorias para evaluar
num_test_images = 5
test_images = np.random.choice(dataset_paths, num_test_images)

# Almacenamos los resultados de las métricas
results = {
    'psnr_bicubic': [],
    'ssim_bicubic': [],
    'psnr_srcnn': [],
    'ssim_srcnn': []
}

# Evaluamos cada imagen
for i, img_path in enumerate(test_images):
    # Aplicamos super-resolución
    original, bicubic, srcnn = apply_srcnn(model, img_path)
    
    # Visualizamos los resultados
    img_name = Path(img_path).stem
    psnr_bic, ssim_bic, psnr_sr, ssim_sr = visualize_results(
        original, bicubic, srcnn, 
        title=f"Imagen de prueba {i+1}: {img_name}"
    )
    
    # Guardamos los resultados
    results['psnr_bicubic'].append(psnr_bic)
    results['ssim_bicubic'].append(ssim_bic)
    results['psnr_srcnn'].append(psnr_sr)
    results['ssim_srcnn'].append(ssim_sr)

In [ ]:
# Calculamos los promedios de las métricas
avg_psnr_bicubic = np.mean(results['psnr_bicubic'])
avg_ssim_bicubic = np.mean(results['ssim_bicubic'])
avg_psnr_srcnn = np.mean(results['psnr_srcnn'])
avg_ssim_srcnn = np.mean(results['ssim_srcnn'])

# Mostramos un resumen de los resultados
print("Resumen de resultados:")
print(f"PSNR promedio - Bicúbica: {avg_psnr_bicubic:.2f}dB, SRCNN: {avg_psnr_srcnn:.2f}dB")
print(f"SSIM promedio - Bicúbica: {avg_ssim_bicubic:.4f}, SRCNN: {avg_ssim_srcnn:.4f}")
print(f"Mejora PSNR: {avg_psnr_srcnn - avg_psnr_bicubic:.2f}dB")
print(f"Mejora SSIM: {avg_ssim_srcnn - avg_ssim_bicubic:.4f}")

# Visualizamos los resultados en un gráfico de barras
plt.figure(figsize=(12, 6))

# Gráfico para PSNR
plt.subplot(1, 2, 1)
plt.bar(['Bicúbica', 'SRCNN'], [avg_psnr_bicubic, avg_psnr_srcnn], color=['blue', 'green'])
plt.title('PSNR Promedio')
plt.ylabel('PSNR (dB)')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Gráfico para SSIM
plt.subplot(1, 2, 2)
plt.bar(['Bicúbica', 'SRCNN'], [avg_ssim_bicubic, avg_ssim_srcnn], color=['blue', 'green'])
plt.title('SSIM Promedio')
plt.ylabel('SSIM')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

### Análisis de Zonas Específicas

Una de las ventajas de los modelos de super-resolución como SRCNN es su capacidad para reconstruir detalles finos en las imágenes. A continuación, analizaremos zonas específicas de algunas imágenes para observar más de cerca cómo el modelo reconstruye estos detalles.

Este análisis nos permitirá apreciar mejor las diferencias entre la interpolación bicúbica simple y la reconstrucción mediante SRCNN, especialmente en áreas con texturas complejas, bordes y detalles finos.

#### Función `visualize_detail()`
Visualiza una región específica de las imágenes con zoom para analizar detalles.
    
**Args**:
- original: Imagen original de alta resolución
- bicubic: Imagen de baja resolución (interpolación bicúbica)
- srcnn: Imagen reconstruida por SRCNN
- region: Tupla (x, y, ancho, alto) que define la región a visualizar. Si es None, se selecciona una región central
- zoom: Factor de zoom para la visualización

In [ ]:
def visualize_detail(original, bicubic, srcnn, region=None, zoom=3):
    # Si no se especifica una región, seleccionamos el centro de la imagen
    if region is None:
        h, w = original.shape[:2]
        size = min(h, w) // 4
        x = (w - size) // 2
        y = (h - size) // 2
        region = (x, y, size, size)
    
    x, y, w, h = region
    
    # Recortamos las regiones
    original_crop = original[y:y+h, x:x+w]
    bicubic_crop = bicubic[y:y+h, x:x+w]
    srcnn_crop = srcnn[y:y+h, x:x+w]
    
    # Configuramos la figura
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Primera fila: imágenes completas con rectángulo marcando la región
    axes[0, 0].imshow(original)
    axes[0, 0].set_title('Original')
    axes[0, 0].add_patch(plt.Rectangle((x, y), w, h, edgecolor='red', linewidth=2, fill=False))
    
    axes[0, 1].imshow(bicubic)
    axes[0, 1].set_title('Bicúbica')
    axes[0, 1].add_patch(plt.Rectangle((x, y), w, h, edgecolor='red', linewidth=2, fill=False))
    
    axes[0, 2].imshow(srcnn)
    axes[0, 2].set_title('SRCNN')
    axes[0, 2].add_patch(plt.Rectangle((x, y), w, h, edgecolor='red', linewidth=2, fill=False))
    
    # Segunda fila: regiones ampliadas
    axes[1, 0].imshow(original_crop)
    axes[1, 0].set_title('Original (Detalle)')
    
    axes[1, 1].imshow(bicubic_crop)
    axes[1, 1].set_title('Bicúbica (Detalle)')
    
    axes[1, 2].imshow(srcnn_crop)
    axes[1, 2].set_title('SRCNN (Detalle)')
    
    # Desactivamos los ejes
    for ax in axes.flatten():
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# Seleccionamos una imagen aleatoria para analizar en detalle
detail_img_path = np.random.choice(dataset_paths)
original, bicubic, srcnn = apply_srcnn(model, detail_img_path)

# Visualizamos la imagen completa
visualize_results(original, bicubic, srcnn, title=f"Análisis detallado: {Path(detail_img_path).stem}")

# Visualizamos una región con detalles
visualize_detail(original, bicubic, srcnn)

# Opcionalmente, podemos seleccionar manualmente una región con detalles interesantes
# Por ejemplo, si detectamos una región con textura o bordes complejos
h, w = original.shape[:2]
# Ejemplo: región en la esquina superior izquierda
region_corner = (0, 0, w//4, h//4)
visualize_detail(original, bicubic, srcnn, region=region_corner)